<table>
  <tr>
    <th>
      <img src="https://drive.google.com/uc?id=1GLaeRA1-wxlBJZITccZTLOkYKK5Z78gS", height = "125", alt="RI-INBRE Logo">
    </th>
    <th>
    <img src="https://drive.google.com/uc?id=1iHm7uZymx9IrwWQnCoUWtpGvdHVLZBFA", height = "125", width="500", alt="MIC Logo">
    </th>
  </tr>
</table>

# Analysis of Biomedical Data for Biomarker Discovery
## Submodule 8: Identification of IRI Biomarkers from Proteomic Data
### Dr. Christopher L. Hemme
### Director, [RI-INBRE Molecular Informatics Core](https://web.uri.edu/riinbre/mic/)
### The University of Rhode Island College of Pharmacy
Last Updated: May 18, 2023

---

## Introduction

In this module we will complete our analysis of the IRI proteomics data by caring out a linear regression based differential analysis (also called differential expression analysis) using the <b>limma</b> package in BioConductor.  In differential analysis, we are looking for significant changes in specific features between one or more states.  We measure these changes by comparing the "peaks" of each feature across the states.  A peak represents whatever signal we're measuring.  For proteomics and metabolomics, this is usually the signal intensity from a mass spectrometry machine.  For next-gen sequencing experiments, these peaks will be based on the read counts from your alignment step.  A peak can represent a variety of signals depending on your experiment, but what matters is that the peak represents some biological quantity.

In comparing two peaks for the same feature across two states, we are interested in whether the difference in peak heights are statistically significant (measured by the <i>p</i>-value) and the magnitude of this difference.  This second value is a measure of the <b>effect size</b> and in omics studies typically the log2(fold change) between the two samples.  We can then use <i>p</i>-value and log2(fold change) cutoffs to identify which proteins significantly differ between states.  While this may sound complicated, to get these values we use the same linear regression methods we studied in Chapter 3 and the coefficients of our regression model are the log2(fold change) values.

<div class="alert alert-block alert-info">
<b>&#9995; Tip:</b> Blue boxes will indicate helpful tips.</div>

<div class="alert alert-block alert-warning">
<b>&#127891; Fun Facts:</b> Used for interesting asides or notes.
</div>

<div class="alert alert-block alert-success">
<b>&#128074; Success:</b> This alert box indicates a successful or positive action.
</div>

<div class="alert alert-block alert-danger">
<b>&#128721; Caution:</b> A red box indicates potential hazards or pitfalls you may encounter.
</div>

---

### Data Prep

We will start with our usual data prep (installation of packages, loading of experimental object, definition of variables).

In [ ]:
if(!require(devtools)) install.packages("devtools")
    devtools::install_github("cran/statmod")

if (!require("BiocManager", quietly = TRUE))
    install.packages("BiocManager")
library('BiocManager')
BiocManager::install("preprocessCore")
BiocManager::install("preprocessCore", configure.args="--disable-threading", force = TRUE)

bioc_packages <- c("limma", "ComplexHeatmap", "M3C", "EnhancedVolcano", "Glimma")
installed_bioc_packages <- bioc_packages %in% rownames(installed.packages())
if (any(installed_bioc_packages == FALSE)) {BiocManager::install(bioc_packages[!installed_bioc_packages])}

packages <- c("plyr", "dplyr", "matrixStats", "statmod")
installed_packages <- packages %in% rownames(installed.packages())
if (any(installed_packages == FALSE)) {install.packages(packages[!installed_packages])}

In [ ]:
require('preprocessCore')
require('limma')
require('Glimma')
require('ComplexHeatmap')
require("EnhancedVolcano")
require('M3C')
require('tidyverse')
require('plyr')
require('matrixStats')
require('statmod')

In [ ]:
exp_obj <- readRDS(file = "data/exp_obj.rds")

<div class="alert alert-block alert-info">
<b>&#9995; Tip:</b> The proteome and metadata objects loaded below are saved in the .rds object in previous submodules. If you see blank variables for either, make sure you've run through the code in submodule 6 and 7 to have the most up to date saved variables in <i>exp_obj.rds</i></div>

In [ ]:
proteome <- exp_obj$data$proteomics$norm
metadata <- exp_obj$metadata
metadata

In [ ]:
#Preview proteome data
head(proteome)

<div class="alert alert-block alert-danger">
<b>&#128721; Caution:</b> Reminder that this version of the experimental object should contain the quantile normalized data from the last chapter.  Use <i>str(exp_obj)</i> if you need to verify that this is the case. 
</div>

<div class="alert alert-block alert-info">
<b>&#9995; Tip:</b> Reminder:
    <ul>
        <li><b>CTRL</b> - Control samples, no surgery or treatment
        <li><b>SHAM</b> - Surgical treatment only; Designed to see if the stress of cutting the rat causes any changes
        <li><b>PLB</b> - IRI model with placebo
        <li><b>TRE</b> - IRI model treated with trepostrinil
    </ul>
</div>

For the sake of simplicity and because this experiment wasn't explicitly designed as a longitudinal study, we will combine our Treatment and Time covariates into a single covariate with 16 factors using a custom R function that we will call <i>group_metadata</i>.  This will allow us to run a simple linear regression model using both covariates.  However, you are welcome to test more complicated models such as a multivariate model with both Treatment and Time as separate covariates.

In [ ]:
# custom function that accepts a metadata data frame and covariates to group together into a single covariate.  
# Returns a new version of the metadata data frame with the new covariate column called Combined added.

group_metadata <- function(metadata, groups) {
  metadata <- metadata %>%
    group_by_at(groups) %>%
    unite(Combined, all_of(groups), sep = "", remove = FALSE)
  metadata$Combined <- factor(metadata$Combined)
  metadata
}
metadata <- group_metadata(metadata, c("Treatment", "Time"))
metadata

Before we run the combined covariates model, let's start with just Treatment.  We will use the cell means version of the formula which sets the intercept to 0.  Because our data is log2 transformed, our regression coefficients will represent the log2 fold change of the given protein between groups.  First we need to build a model matrix which takes our model and converts the categorical variables to dummy variables (0 or 1). Our model matrix will be as follows:

In [ ]:
molecule.model.treatment <- model.matrix(~ 0 + Treatment, metadata)
colnames(molecule.model.treatment) <- levels(metadata$Treatment)
molecule.model.treatment

<div class="alert alert-block alert-info">
<b>&#9995; Tip:</b> The model matrix is also called a design matrix.</div>

The next step will be to define the parameters of our regression model.  We will account for the batch effect observed in the last chapter by setting the <i>Batch</i> column as a blocking factor in the function <i>duplicateCorrelation</i> which computes the intrablock correlation.  Once these parameters are set, we will run a simple fit which <b>limma</b> will return as an <b>MArrayLM</b> object.  Because we need to account for our blocking factor, we will first run the <i>duplicateCorrelation</i> function to estimate the the correlations between the observations of our blocking variable (i.e. our batch effect).  We will then pass these results to the <i>lmFit</i> function.

In [ ]:
molecule.corfit.treatment <- duplicateCorrelation(
    proteome,
    molecule.model.treatment,
    block = metadata$Batch
)

In [ ]:
molecule.SimpleFit.treatment <- lmFit(
    proteome,
    molecule.model.treatment,
    block = metadata$Batch,
    correlation = molecule.corfit.treatment$consensus
)
molecule.SimpleFit.treatment

We'll look at these results in more detail later.  Right now, we need to think about <b>contrasts</b>.  Typically when we want to compare two states, we would use a simple <b>t-test</b>.  However, using multiple t-tests can introduce error if it's not corrected.  A better strategy is to extract pairwise information from our global regression model using <b>contrasts</b>.  With contrasts, we can define very sophisticated comparisons by combining different factors together, but we'll keep it simple for now and just compare different treatment states.

In [ ]:
# define contrasts
contrasts.treatment <- c("SHAM-CTRL", "PLB-SHAM", "TRE-PLB")
# make the contrasts
molecule.contrasts.treatment <- makeContrasts(contrasts = contrasts.treatment, levels = molecule.model.treatment)
# refit the simple fit we previously ran using the contrast data
molecule.contrastsFit.treatment <- contrasts.fit(molecule.SimpleFit.treatment, molecule.contrasts.treatment)
# calculate stats and rank proteins using an emperical Bayes method
molecule.contrastsFit.treatment <- eBayes(molecule.contrastsFit.treatment)
# fix column names to match contrasts
colnames(molecule.contrastsFit.treatment$coefficients) = contrasts.treatment

The full regression model gives us a global overview of which covariates and features are important.  With contrasts, we can break that information down and see the changes for each protein within the context of each specific pair of Treatment states we're interested in.

In [ ]:
molecule.contrastsFit.treatment$coefficients[1:20,]

Before we analyze the specific protein changes, let's look at some diagnostic plots.  We'll start with the <b>MA plot</b>.  The MA plot shows the relationship between the log2(fold change) (the "M" in MA) and the mean expression (the "A").  We expect most proteins will not show differential expression in a given experiment, and thus most of the data points will be located at 0.  Extreme values along the y-axis indicate proteins with strongly differential values between the two states.  Particularly when working with genomic data, the MA plot will often show a "fanning" effect due to the fact that low expression genes (plotted on the left) show more variability in fold change than highly expressed genes (plotted on the right).  MA plots are also useful for evaluating the effects of normalization on the data.  Let's look at the MA plots for our three contrasts to get a sense of the landscape of our data.  We'll use the <b>Glimma</b> package which is used to create interactive plots from <b>limma</b> data.  You will be able to clock on specific data points to see the effects on specific genes.

<div class="alert alert-block alert-info">
<b>&#9995; Tip:</b> Glimma plots are interactive.  If you click on datapoints on the graph, the names of the points will appear in the box below and filter the table.</div>

In [ ]:
#SHAM-CTRL
glimmaMA(molecule.contrastsFit.treatment, coef=1)

In [ ]:
#PLB-SHAM
glimmaMA(molecule.contrastsFit.treatment, coef=2)

In [ ]:
#TRE-PLB
glimmaMA(molecule.contrastsFit.treatment, coef=3)

What do these results show us?  As hoped, there aren't many significant difference between the expression profiles of control and sham, meaning the surgical technique itself introduces a minimal amount of noise.  As expected, the IRI itself induces many proteomic changes compared to the sham.  Based on treatment alone though, there are few significant changes in protein levels for the treated state.  However, we still have to consider the time component.  In early time points, the changes may not be significant enough to see an effect from treatment, and in later time points the damage could be significant enough that treatment is ineffective.  To get a better sense of what is happening, we will need to add the time component.  First though, let's verify this with the volcano plots.

A <b>volcano plot</b> is another extremely useful visualization tool for comparing pairwise omics data.  In this case, we are plotting the log2(fold change) on the x-axis and the -log10(<i>p</i>-value) on the y-axis.  This centers the data on the x-axis on zero, meaning that the plot on each side of zero represents one of our factors of interest.  By applying cutoff, we can easily identify quadrants of significantly different proteins and easily identify which treatment state they represent.

In [ ]:
#SHAM-CTRL
glimmaVolcano(molecule.contrastsFit.treatment, coef=1)

In [ ]:
#PLB-SHAM
glimmaVolcano(molecule.contrastsFit.treatment, coef=2)

In [ ]:
#TRE-PLB
glimmaVolcano(molecule.contrastsFit.treatment, coef=3)

The results are consistent but easier to visualize on the volcano plot.  It's important to remember that the volcano plot is showing <b>relative</b> abundance, that is, how abundant is a signal in one condition versus another.  It doesn't tell you anything about the expression levels themselves.  The MA plot and box plots (and the raw data of course) are useful for determining why this abundance occurs (e.g. a protein is more highly expressed in condition B vs. condition A).

Now we will use the <i>topTable</i> function to look at the regression coefficients themselves for SHAM-CTRL, PLB_SHAM, and finally TRE-PLB. 

In [ ]:
topTable(molecule.contrastsFit.treatment, coef = "SHAM-CTRL", number = 10, adjust.method = "BH", sort.by = "M")

In [ ]:
topTable(molecule.contrastsFit.treatment, coef = "PLB-SHAM", number = 10, adjust.method = "BH", sort.by = "M")

In [ ]:
topTable(molecule.contrastsFit.treatment, coef = "TRE-PLB", number = 10, adjust.method = "BH", sort.by = "M")

<div class="alert alert-block alert-info">
    <b>&#9995; Tip:</b> The "M" in the <b>sort.by</b> argument refers to the log2(fold change), or the "M" axis in our MA plot.</div>

The cumulative results show that there is a clear change in state between the sham and the placebo (i.e. untreated IRI) states, but we will need to include the time component to better understand what is happening.  Let's run these analyses again, except this time we will use our combined covariate to account for time. We'll only look at the 6 hour comparisons but feel free to try different contrasts. The following code is the same as above except for the formula used to define our model. Above, we used this formula to define our model as <i>model.matrix(\~0 + Treatment, metadata)</i> whereas this below code defines the model as <i>model.matrix(\~0 + Treatment, metadata).</i>

In [ ]:
molecule.model.combined <- model.matrix(~ 0 + Combined, metadata)
colnames(molecule.model.combined) <- levels(metadata$Combined)
molecule.model.combined

In [ ]:
molecule.corfit.combined <- duplicateCorrelation(
    proteome,
    molecule.model.combined,
    block = metadata$Batch
)

In [ ]:
molecule.SimpleFit.combined <- lmFit(
    proteome,
    molecule.model.combined,
    block = metadata$Batch,
    correlation = molecule.corfit.combined$consensus
)
molecule.SimpleFit.combined

In [ ]:
contrasts.combined <- c("SHAM6-CTRL0", "PLB6-SHAM6", "TRE6-PLB6")
molecule.contrasts.combined <- makeContrasts(contrasts = contrasts.combined, levels = molecule.model.combined)
molecule.contrastsFit.combined <- contrasts.fit(molecule.SimpleFit.combined, molecule.contrasts.combined)
molecule.contrastsFit.combined <- eBayes(molecule.contrastsFit.combined)
colnames(molecule.contrastsFit.combined$coefficients) = contrasts.combined

In [ ]:
molecule.contrastsFit.combined$coefficients[1:20,]

In [ ]:
#SHAM6-CTRL0
glimmaMA(molecule.contrastsFit.combined, coef=1)

In [ ]:
#PLB6-SHAM6
glimmaMA(molecule.contrastsFit.combined, coef=2)

In [ ]:
#TRE6-PLB6
glimmaMA(molecule.contrastsFit.combined, coef=3)

In [ ]:
#SHAM6-CTRL0
glimmaVolcano(molecule.contrastsFit.combined, coef=1)

In [ ]:
#PLB6-SHAM6
glimmaVolcano(molecule.contrastsFit.combined, coef=2)

In [ ]:
#TRE6-PLB6
glimmaVolcano(molecule.contrastsFit.combined, coef=3)

In [ ]:
topTable(molecule.contrastsFit.combined, coef = "SHAM6-CTRL0", number = 10, adjust.method = "BH", sort.by = "M")

In [ ]:
topTable(molecule.contrastsFit.combined, coef = "PLB6-SHAM6", number = 10, adjust.method = "BH", sort.by = "M")

In [ ]:
topTable(molecule.contrastsFit.combined, coef = "TRE6-PLB6", number = 10, adjust.method = "BH", sort.by = "M")

<div class="alert alert-block alert-info">
<b>&#9995; Tip:</b> As an exercise, redo the previous analysis using the multivariate model "~ 0 + Treatment + Time" to see if separating the covariates has an affect on the results.</div>

Now we have a clearer view of the changes that are happening between the states.  At 6 hours when we expect IRI damage to be noticeable but still treatable, we start to see significant differences in the protein abundances between the sham and placebo states, but we also see several proteins affected by the trepostrinil treatment.  While we can try different models to identify other covariates affected the results, we have now identified a set of potential biomarkers that can indicate the degree of severity of IRI and other biomarkers which may indicate the effects of the treatment on IRI damage.

---

# Finishing Our Experiment

You should now be familiar with the concepts of linear and logistic regression and how they can be used to compare biomarkers or to classify samples or to assess the quality of biomarkers.  We have discussed common methods of exploratory analysis such as PCA and heatmaps that are useful for identifying global patterns in omics data.  Finally, we discussed differential analysis of omics data to identify specific potential biomarkers.  However, we are ending on something of a cliffhanger.  Now that we have a list of potential protein biomarkers from our proteomics analysis, what comes next?

First, we need to assess the biological relevance of our results.  A common way to do this is with <b>pathway analysis</b>.  Most proteins are part of complex metabolic and regulatory networks that represent some biological theme (e.g. TCA cycle, apoptosis, etc.).  We would expect proteins within a specific pathway or biological process to be correlated together, otherwise it could lead to significant disruption of the cellular machinery.  Just as we can test for differential abundance of proteins, we can also look at differential abundance of pathways.  In our case, several of the proteins we find deferentially expressed are related to kidney injury and function, which is what we would expect.  With this biological knowledge, we can begin to filter our list of potential biomarkers to include thode that are most likely to be biological relevant in our model (i.e. causative of or directly caused by the state change).

Ideally, we would design our experiments to have sufficient numbers of samples and controls to maximize the statistical power of the experiment.  In practice, budget constraints and other factors often limit the number of samples and replicates we can test.  A complementary approach is to compare your data set with similar datasets from other sources to maximize the statistical power of the grouped data set.  This is known as <b>meta-analysis</b>. Meta-analysis is particularly useful today because the number of omics datasets is very high compared to the past and for commonly studied systems, it's possible to have dozens if not hundreds of related datasets.  However, meta-analysis is not trivial.  In our single proteomics dataset, we identified one batch effect, but in meta-analysis, each dataset is likely to have its own unique batch effects.  These can result from experimental design, sample prep strategy, instrument used to generate the data, the skills of the lab staff, geographical location, and numerous other sources.  Because of the complexity of meta-analysis, many labs choose not to do this.  When properly performed however, meta-analysis can provide an additional level of confidence that the biomarkers you have identified are biologically and statistically significant.

One common criticism of omics data is whether the results are real or a result of experimental artifacts. As a data set gets bigger, it becomes more likely that you will begin to find statistically significant results purely by chance.  To forestall such criticisms, it is standard procedure to experimentally validate key omics results whenever possible.  In our case, that means testing the potential biomarkers experimentally to prove that the patterns we see are actually happening.  There are several ways to experimentally validate omics results, the most common of which are listed below:

| Omics Method | Experimental Validation Method |
| --- | --- |
| Transcriptomics | Quantitative PCR, Northern Blot |
| Epigenomics | ChIP, Protein-Binding Assay |
| Proteomics | Western Blot |
| Metabolomics/Lipidomics/Glycomics | HPLC, NMR |

Once validated with experimental results, we can begin comparing our potential biomarker to known biomarkers using the regression methods we have already discussed.  More sophisticated methods of biomarker validation can be found in the scientific literature, and you should now have the background to study these methods on your own.

To summarize, this would be the strategy we would follow for identifying potential proteomic biomarkers:

1. Generate hypothesis designed to address your biological question
2. Design proteomics experiment with proper controls and identify relevant covariates
3. Generate proteomics data
4. Differential analysis of proteomics data to identify potential biomarkers indicating relevant state changes
5. Pathway analysis to link potential biomarkers to pathways relevant to your hypothesis and to identify differentially expressed pathways
6. Experimental validation of biologically relevant biomarkers using Western blots
7. Comparison of potential biomarkers to known biomarkers to assess clinical utility

## But Wait, There's More!

For a traditional bioinformatics analysis, this would be the end of this particular experiment.  However, at the time of this writing, the field of bioinformatics is undergoing a rapid transition.  We now have publicly available omics datasets from thousands of laboratories spanning 20+ years.  We are also rapidly trasitioning to <b>single cell</b> and <b>spatial omics</b> methods which allow for much much higher resolution omics studies than traditionally available.  This obviously represents a huge amount of data and analysis by traditional methods is becoming difficult.  This is precipitating a transition of biomedical research to cloud environments coupled to <b>machine learning</b> techniques designed to handle massive, complex data sets.  While this may sound daunting, we've already covered basic techniques involved in machine learning, including linear and logistics regression, principle components analysis, and clustering.  While there are many applications for machine learning in biomedical data science, biomarker discovery is especially amenible to these methods and thus we will introduce basic ML models for biomarker discovery in the final submodule, <b>Submodule 9: Biomarker Discovery Using Machine Learning</b>.  

---

<p><span style="font-size: 30px"><b>Quizzes</b></span> <span style="float : inline;">(run the command below to display the quizzes)</span> </p>

In [ ]:
IRdisplay::display_html('<iframe src="quizes/Chapter8_Quizes.html" width=100% height=450></iframe>')

---

## References

[Hou J, Tolbert E, Birkenbach M, Ghonem NS. Treprostinil alleviates hepatic mitochondrial injury during rat renal ischemia-reperfusion injury. Biomed Pharmacother. 2021 Nov;143:112172. doi: 10.1016/j.biopha.2021.112172. Epub 2021 Sep 21. PMID: 34560548; PMCID: PMC8550798.][hou]<br>
[Ding M, Tolbert E, Birkenbach M, Gohh R, Akhlaghi F, Ghonem NS. Treprostinil reduces mitochondrial injury during rat renal ischemia-reperfusion injury. Biomed Pharmacother. 2021 Sep;141:111912. doi: 10.1016/j.biopha.2021.111912. Epub 2021 Jul 15. PMID: 34328097; PMCID: PMC8429269.][ding]<br>

[ding]: https://pubmed.ncbi.nlm.nih.gov/34328097/ "Ding M, Tolbert E, Birkenbach M, Gohh R, Akhlaghi F, Ghonem NS. Treprostinil reduces mitochondrial injury during rat renal ischemia-reperfusion injury. Biomed Pharmacother. 2021 Sep;141:111912. doi: 10.1016/j.biopha.2021.111912. Epub 2021 Jul 15. PMID: 34328097; PMCID: PMC8429269."
[hou]: https://pubmed.ncbi.nlm.nih.gov/34560548/ "Hou J, Tolbert E, Birkenbach M, Ghonem NS. Treprostinil alleviates hepatic mitochondrial injury during rat renal ischemia-reperfusion injury. Biomed Pharmacother. 2021 Nov;143:112172. doi: 10.1016/j.biopha.2021.112172. Epub 2021 Sep 21. PMID: 34560548; PMCID: PMC8550798."

---